# Stacks and queues
A stack has a last in first out structure (LIFO), like a can of pringles. A queue has a first in first out (FIFO) structure. Appending to and popping from are standard with dynamic arrays in python and use $O(1)$ amortized time. A queue struction may require use of the deque collection, as inserting at the start of a dynamic array requires $O(n)$ time.

#### Compress array
Given an array of integers, perform (repeatedly) a compress operation that finds the first consecutive equal numbers and combines them as their sum.

In [3]:
# Rather than constantly recomposing the array (with each alteration taking O(n) time), we create a stack. 
# This allows us to add to or remove from the end with O(n) time. Space complexity is O(n).

def compress_array(arr):
    n = len(arr)
    if n < 2:
        return arr
    stack = []
    i = 0
    while i < len(arr):
        if stack == []:
            stack.append(arr[i])
            i += 1
        elif stack[-1] == arr[i]:
            stack.pop()
            arr[i] += arr[i]
        else:
            stack.append(arr[i])
            i += 1
    return stack
            

In [11]:
arr = [8,4,2,2,2,4]
print(f"Compress {arr}: {compress_array(arr)}")
arr = [4,4,4,4]
print(f"Compress {arr}: {compress_array(arr)}")
arr = [1,2,3,4]
print(f"Compress {arr}: {compress_array(arr)}")

Compress [8, 4, 2, 2, 2, 4]: [16, 2, 4]
Compress [4, 4, 4, 4]: [16]
Compress [1, 2, 3, 4]: [1, 2, 3, 4]


In [9]:
# A more elegant solution...
def compress_array(arr):
    stack = []
    for num in arr:
        while stack and stack[-1] == num:
            num += stack.pop()
        stack.append(num)
    return stack